In [1]:
import pickle
import os
import pandas as pd
import uuid

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [ ]:
year = 2023
month = 6
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata/{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = 'os.getenv('RUN_ID', '________________________')

logged_model = f'runs:/{RUN_ID}/model'

In [3]:
def generate_uuids(n):
    ride_is = []
    for i in range(n):
        ride_is.append(str(uuid.uuid4()))
    return ride_is

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
def apply_model(input_file, model_version, output_)



df = read_dataframe('data/green_tripdata_2021-01.parquet')
dicts = prepare_dictionaries(df)

model = mlflow.pyfunc.load_model(logged_model)
y_pred = model.predict(dicts)

In [ ]:
df['ride_id'] = generate_uuids(len(df))

In [ ]:
df_result  = pd.DataFrame()

In [ ]:
df_result['ride_id'] = df['ride_id']
df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
df_result['PULocationID'] = df['PULocationID']
df_result['DOLocationID'] = df['DOLocationID']
df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
df_result['model_version'] = RUN_ID


In [ ]:
df_result

In [ ]:
!mkdir output

In [ ]:
df_result.to_parquet(output_file)